# 实验 10：通过 SageMaker Studio 使用 SageMaker 管道和 SageMaker 模型注册表

在本实验中，您将创建并运行 Amazon Sagemaker 管道以及监控管道进度。您还将找到并浏览机器学习 (ML) 流程使用或生成的一些构件。

如时间允许，您还能查看管道生成的模型所沿袭的详细信息。

## 任务 2.1 环境设置

在创建 SageMaker 管道前，必须通过安装必要程序包、导入模块和暂存支持文件来准备环境。该管道旨在使用特征组，因此您还要在 Amazon SageMaker 特征存储中创建特征组并运行 Data Wrangler 流，以便准备环境。

运行该任务中的单元格以执行以下操作：
- 安装依赖项。
- 导入所需模块。
- 将数据和代码复制到 Amazon Simple Storage Service (Amazon S3)。
- 创建特征组。
- 将特征摄取到特征组。

### 任务 2.1.1 安装依赖项

In [ ]:
#install dependencies
%pip install --upgrade pip 
%pip install pytest-astropy ==  0.7.0
%pip install rsa == 4.7.2
%pip install PyYAML
!apt update && apt install -y git
%pip install git+https://github.com/aws-samples/ml-lineage-helper

### 任务 2.1.2 导入模块

In [ ]:
#import-modules
import os
import json
import boto3
import sagemaker
import sagemaker_datawrangler
import sagemaker.session
import datetime as dt
import pandas as pd
import time
from time import gmtime, strftime
import uuid
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model import Model
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
from sagemaker.transformer import Transformer
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from ml_lineage_helper import *
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.session import Session
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor
from sagemaker.network import NetworkConfig
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

In [ ]:
#create sessions
boto_session  =  boto3.Session()
sagemaker_session = sagemaker.Session()


In [ ]:
#create clients
s3_client = boto3.client('s3')
featurestore_runtime = boto3.client('sagemaker-featurestore-runtime')
sagemaker_client = boto3.client('sagemaker')

In [ ]:
#feature store session
feature_store_session = Session(
    boto_session = boto_session,
    sagemaker_client = sagemaker_client,
    sagemaker_featurestore_runtime_client = featurestore_runtime
)

In [ ]:
#set global variables
default_bucket = sagemaker_session.default_bucket()
region = boto_session.region_name
role = sagemaker.get_execution_role()

### 任务 2.1.3 将实验文件复制到 Amazon S3 

In [ ]:
# Upload files to default bucket
s3_client.put_object(Bucket = default_bucket, Key = 'data/')
s3_client.put_object(Bucket = default_bucket, Key = 'input/code/')
s3_client.upload_file('pipelines/data/storedata_total.csv', default_bucket, 'data/storedata_total.csv')
s3_client.upload_file('pipelines/input/code/evaluate.py', default_bucket, 'input/code/evaluate.py')
s3_client.upload_file('pipelines/input/code/generate_config.py', default_bucket, 'input/code/generate_config.py')
s3_client.upload_file('pipelines/input/code/processfeaturestore.py', default_bucket, 'input/code/processfeaturestore.py')

# Preview the dataset
print('Dataset preview:')
customer_data = pd.read_csv('pipelines/data/storedata_total.csv')
customer_data.head()

### 任务 2.1.4 创建特征组

在此任务中，您将为数据创建一个特征组。首先，创建一个数据架构。在本实验中，应首先按 **name**（名称）列对架构进行排序，然后按变量的 **type**（类型）列进行排序。

In [ ]:
#set-up-feature-store-variables
record_identifier_feature_name = 'FS_ID'
event_time_feature_name = 'FS_time'

column_schemas = [
    {
        "name": "retained",
        "type": "long"
    },
    {
        "name": "esent",
        "type": "long"
    },
    {
        "name": "eopenrate",
        "type": "float"
    },
    {
        "name": "eclickrate",
        "type": "float"
    },
    {
        "name": "avgorder",
        "type": "float"
    },
    {
        "name": "ordfreq",
        "type": "float"
    },
    {
        "name": "paperless",
        "type": "long"
    },
    {
        "name": "refill",
        "type": "long"
    },
    {
        "name": "doorstep",
        "type": "long"
    },
    {
        "name": "first_last_days_diff",
        "type": "long"
    },
    {
        "name": "created_first_days_diff",
        "type": "long"
    },
    {
        "name": "favday_Friday",
        "type": "long"
    },
    {
        "name": "favday_Monday",
        "type": "long"
    },
    {
        "name": "favday_Saturday",
        "type": "long"
    },
    {
        "name": "favday_Sunday",
        "type": "long"
    },
    {
        "name": "favday_Thursday",
        "type": "long"
    },
    {
        "name": "favday_Tuesday",
        "type": "long"
    },
    {
        "name": "favday_Wednesday",
        "type": "long"
    },
    {
        "name": "city_BLR",
        "type": "long"
    },
    {
        "name": "city_BOM",
        "type": "long"
    },
    {
        "name": "city_DEL",
        "type": "long"
    },
    {
        "name": "city_MAA",
        "type": "long"
    },
    {
        "name": "FS_ID",
        "type": "long"
    },
    {
        "name": "FS_time",
        "type": "float"
    }
]


现在，创建特征组。

In [ ]:
# Flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = 'featureengineer'
flow_export_id = f"{strftime('%d-%H-%M-%S', gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"

# Feature group name, with flow_name and a unique id. You can give it a customized name
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"

# SageMaker Feature Store writes the data in the offline store of a Feature Group to a 
# Amazon S3 location owned by you.
feature_store_offline_s3_uri = 's3://' + default_bucket

# Controls if online store is enabled. Enabling the online store allows quick access to 
# the latest value for a record by using the GetRecord API.
enable_online_store = True

In [ ]:
#create-feature-group
default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name = column_schema['name'], 
        feature_type = column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]


print(f"Feature Group Name: {feature_group_name}")

# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup = 'primary',
        ConfigurationUpdates = {
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

feature_group = FeatureGroup(
    name = feature_group_name, sagemaker_session = feature_store_session, feature_definitions = feature_definitions)

feature_group.create(
    s3_uri = feature_store_offline_s3_uri,
    record_identifier_name = record_identifier_feature_name,
    event_time_feature_name = event_time_feature_name,
    role_arn = role,
    enable_online_store = enable_online_store
)

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for feature group creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"Feature Group {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group = feature_group)


### 任务 2.1.5 摄取特征

完成此过程大约需要 8 分钟。

In [ ]:
#populate-feature-store
column_list = ['retained','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless','refill','doorstep','first_last_days_diff','created_first_days_diff','favday_Friday','favday_Monday', 'favday_Saturday','favday_Sunday','favday_Thursday','favday_Tuesday','favday_Wednesday','city_BLR','city_BOM','city_DEL','city_MAA','FS_ID','FS_time']
lab_test_data = pd.read_csv('featureengineer_data/store_data_processed.csv', names = (column_list), header = 1)
feature_group.ingest(data_frame = lab_test_data, wait = True)

## 任务 2.2 创建并运行 SageMaker 管道

请注意，设置完环境，您将配置、创建并开启 SageMaker 管道。

SageMaker 管道是运行一组依赖步骤的工作流。这些步骤可以接收输入并发送输出，因此数据和其他资产可以在它们之间进行传递。

运行以下单元格，执行以下操作：
- 定义配置管道所需的变量。
- 配置 SageMaker 会话。
- 定义管道步骤。
- 配置管道。
- 创建管道。
- 开启管道。
- 描述管道。
- 创建等待事件，让笔记本等到管道完成运行后再继续。

### 任务 2.2.1 设置管道使用的变量

In [ ]:

#pipeline-variables
feature_group_name = feature_group.name
model_name = "Churn-model"

sklearn_processor_version = "0.23-1"
model_package_group_name = "ChurnModelPackageGroup"
pipeline_name = "ChurnModelSMPipeline"

processing_instance_count = ParameterInteger(
    name = "ProcessingInstanceCount",
    default_value = 1
    )

processing_instance_type = ParameterString(
        name = "ProcessingInstanceType",
        default_value = "ml.m5.xlarge"
    )

training_instance_type = ParameterString(
        name = "TrainingInstanceType",
        default_value = "ml.m5.xlarge"
    )

input_data = ParameterString(
        name = "InputData",
        default_value = "s3://{}/data/storedata_total.csv".format(default_bucket), 
    )

batch_data = ParameterString(
        name = "BatchData",
        default_value = "s3://{}/data/batch/batch.csv".format(default_bucket),
    )

### 任务 2.2.2 配置管道

定义名为 **ChurnModelPipeline** 的管道，以生成可评估客户保留和流失可能性的模型。此管道有九个步骤。

管道中的每个步骤均运行特定的任务类型。任务需要的输入因任务类型而异。有关 SageMaker 管道步骤类型的更多信息，请参阅[步骤类型](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#build-and-manage-steps-types)。

查看以下单元格中的代码，了解每个步骤的定义方法：

**ChurnModelProcess** 步骤是在名为 **step_process** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Processing（处理）- 处理任务使用类 ProcessingStep() 进行定义。
- **Processor**（处理程序）：SKLearnProcessor。
- **Destination**（目的地）：输出将发送至默认 S3 存储桶下的文件夹。
- **Job Arguments**（任务参数）：此步骤将使用特征存储来处理数据集。
- **Code**（代码）：位于默认 S3 存储桶中的 **processfeaturestore.py**。


In [ ]:
#configure-processing-step
# Run a scikit-learn script to do data processing on SageMaker 
# using the SKLearnProcessor class
sklearn_processor = SKLearnProcessor(
        framework_version = sklearn_processor_version,
        instance_type = processing_instance_type.default_value, 
        instance_count = processing_instance_count,
        sagemaker_session = sagemaker_session,
        role = role,
    )

# Inputs, outputs, and code are parameters to the processor
# step_* will become the pipeline steps toward the end of the cell
# in this case, use the feature store as input, so there is no externalinput
step_process = ProcessingStep(
        name = "ChurnModelProcess",
        processor = sklearn_processor,
        outputs = [
            ProcessingOutput(output_name = "train", source = "/opt/ml/processing/train",\
                             destination = f"s3://{default_bucket}/output/train" ),
            ProcessingOutput(output_name = "validation", source = "/opt/ml/processing/validation",\
                            destination = f"s3://{default_bucket}/output/validation"),
            ProcessingOutput(output_name = "test", source = "/opt/ml/processing/test",\
                            destination = f"s3://{default_bucket}/output/test"),
            ProcessingOutput(output_name = "batch", source = "/opt/ml/processing/batch",\
                            destination = f"s3://{default_bucket}/data/batch"),
            ProcessingOutput(output_name = "baseline", source = "/opt/ml/processing/baseline",\
                            destination = f"s3://{default_bucket}/input/baseline")
        ],
        job_arguments = ["--featuregroupname",feature_group_name,"--default-bucket",default_bucket,"--region",region],
        code = f"s3://{default_bucket}/input/code/processfeaturestore.py",
    )

**ChurnHyperParameterTuning** 步骤是在名为 **step_tuning** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Tuning（优化）– 优化任务使用类 TuningStep() 进行定义。
- **Tuner**（优化程序）：此任务使用 XGBoost 框架。
- **Inputs**（输入）：注意，此任务使用 ChurnModelProcess 步骤 (**step_process**) 生成的训练和验证数据。

In [ ]:
#configure-churn-hyperparameter-tuning
# Training/tuning step for generating model artifacts
model_path = f"s3://{default_bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework = "xgboost",
    region = region,
    version = "1.5-1",
    py_version = "py3",
    instance_type = training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
    }

xgb_train = Estimator(
    image_uri = image_uri,
    instance_type = training_instance_type,
    instance_count = 1,
    hyperparameters = fixed_hyperparameters,
    output_path = model_path,
    base_job_name = f"churn-train",
    sagemaker_session = sagemaker_session,
    role = role
    )

In [ ]:
#Tuning steps
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
    }
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "ChurnHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs = 2, max_parallel_jobs = 2),
    inputs = {
            "train": TrainingInput(
                s3_data = step_process.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type = "text/csv",
            ),
            "validation": TrainingInput(
                s3_data = step_process.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type = "text/csv",
            ),
        },
    )

**ChurnEvalBestModel** 步骤是在名为 **step_eval** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Processing（处理）。
- **Processor**（处理程序）：ScriptProcessor。
- **Inputs**（输入）：注意，此任务使用来自 ChurnHyperParameterTuning (**step_tuning**) 的顶级模型和来自 ChurnModelProcess (**step_process**) 的测试输出。
- **Outputs**（输出）：输出将写入默认 S3 存储桶。
- **Code**（代码）：名为 **evaluate.py** 的脚本（位于 Amazon S3 中）将用于评估。

In [ ]:
#configure-churn-best-model
evaluation_report = PropertyFile(
    name = "ChurnEvaluationReport",
    output_name = "evaluation",
    path = "evaluation.json",
)

script_eval = ScriptProcessor(
    image_uri = image_uri,
    command = ["python3"],
    instance_type = processing_instance_type,
    instance_count = 1,
    base_job_name = "script-churn-eval",
    role = role,
    sagemaker_session = sagemaker_session,
)

step_eval = ProcessingStep(
    name = "ChurnEvalBestModel",
    processor = script_eval,
    inputs = [
        ProcessingInput(
            source = step_tuning.get_top_model_s3_uri(top_k = 0, s3_bucket = default_bucket, prefix = "output"),
            destination = "/opt/ml/processing/model"
        ),
        ProcessingInput(
            source = step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination = "/opt/ml/processing/test"
        )
    ],
    outputs = [
        ProcessingOutput(output_name = "evaluation", source = "/opt/ml/processing/evaluation",\
                            destination = f"s3://{default_bucket}/output/evaluation"),
    ],
    code = f"s3://{default_bucket}/input/code/evaluate.py",
    property_files = [evaluation_report],
)

**ChurnCreateModel** 步骤是在名为 **step_create_model** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Model（模型）– 模型任务使用类 Model() 进行定义。
- **Model**（模型）：步骤使用的模型是在之前定义的名为 **model** 的变量中定义的。注意，**model** 变量使用 ChurnHyperParameterTuning (**step_tuning**) 创建的顶级模型。
- **Inputs**（输入）：输入包含实例类型和加速程序类型。

In [ ]:
#configure-model-creation
model = Model(
    image_uri = image_uri,        
    model_data = step_tuning.get_top_model_s3_uri(top_k = 0,s3_bucket = default_bucket,prefix = "output"),
    name = model_name,
    sagemaker_session = sagemaker_session,
    role = role,
)

inputs = CreateModelInput(
    instance_type = "ml.m5.large",
    accelerator_type = "ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name = "ChurnCreateModel",
    model = model,
    inputs = inputs,
)

**ChurnModelConfigFile** 步骤是在名为 **step_config_file** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Processing（处理）。
- **Processor**（处理程序）：ScriptProcessor。
- **Code**（代码）：位于默认 S3 存储桶的 **generate_config.py**。
- **Job Arguments**（任务参数）：任务参数包括 **ChurnCreateModel** 生成的模型、偏差报告路径、默认存储桶、样本数量以及用于处理的实例数量。
- **Depends On**（前提）：注意，此任务在模型创建完成后才可运行。

In [ ]:

#configure-script-processing
bias_report_output_path = f"s3://{default_bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
analysis_config_path = f"s3://{default_bucket}/clarify-output/bias/analysis_config.json"
clarify_image = sagemaker.image_uris.retrieve(framework = 'sklearn', version = sklearn_processor_version, region = region)

#custom_image_uri = None
script_processor = ScriptProcessor(
    command = ['python3'],
    image_uri = clarify_image,
    role = role,
    instance_count = 1,
    instance_type = processing_instance_type,
    sagemaker_session = sagemaker_session,
)

step_config_file = ProcessingStep(
    name = "ChurnModelConfigFile",
    processor = script_processor,
    code = f"s3://{default_bucket}/input/code/generate_config.py",
    job_arguments = ["--modelname", step_create_model.properties.ModelName, "--bias-report-output-path", bias_report_output_path, "--clarify-instance-type", clarify_instance_type,\
                  "--default-bucket", default_bucket, "--num-baseline-samples", "50", "--instance-count", "1"],
    depends_on = [step_create_model.name]
)

**ChurnTransform** 步骤是在名为 **step_transform** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Transform（转换）- 转换任务使用类 TransformStep() 进行定义。
- **Transformer**（转换程序）：转换程序详细信息是在之前定义的名为 **transformer** 的变量中设置的。注意，此变量使用 ChurnCreateModel (**step_create_model**) 中创建的模型。
- **Inputs**（输入）：要转换的数据 batch.csv 之前已在笔记本中定义。输入还包括文件类型和拆分方法。

In [ ]:
#configure-inference
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type = "ml.m5.xlarge",
    instance_count = 1,
    assemble_with = "Line",
    accept = "text/csv",    
    output_path = f"s3://{default_bucket}/ChurnTransform"
    )

step_transform = TransformStep(
    name = "ChurnTransform",
    transformer = transformer,
    inputs = TransformInput(data = batch_data, content_type = "text/csv", join_source = "Input", split_type = "Line")
    )

**ClarifyProcessingStep** 步骤是在名为 **step_clarify** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Processing（处理）。
- **Processor**（处理程序）：此任务使用 SageMakerClarifyProcessor。您可以在名为 **clarify_processor** 的变量中查看处理程序配置。
- **Inputs**（输入）：输入是在 **data_input** 和 **congif_input** 变量中定义的。
- **Outputs**（输出）：输出将写入默认存储桶下的文件夹。
- **Depends On**（前提）：注意，在 **ChurnModelConfigFile** 已创建 Amazon SageMaker Clarify 需要的配置文件后，此任务才可运行。

In [ ]:
#configure-clarify-processing
data_config = sagemaker.clarify.DataConfig(
s3_data_input_path = f's3://{default_bucket}/output/train/train.csv',
s3_output_path = bias_report_output_path,
    label = 0,
    headers = ['target','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless','refill','doorstep','first_last_days_diff','created_first_days_diff','favday_Friday','favday_Monday','favday_Saturday','favday_Sunday','favday_Thursday','favday_Tuesday','favday_Wednesday','city_BLR','city_BOM','city_DEL','city_MAA'],
    dataset_type = "text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role = role,
    instance_count = 1,
    instance_type = clarify_instance_type,
    sagemaker_session = sagemaker_session,
)

config_input = ProcessingInput(
    input_name = "analysis_config",
    source=analysis_config_path,
    destination = "/opt/ml/processing/input/analysis_config",
    s3_data_type = "S3Prefix",
    s3_input_mode = "File",
    s3_compression_type = "None",
    )

data_input = ProcessingInput(
    input_name = "dataset",
    source = data_config.s3_data_input_path,
    destination = "/opt/ml/processing/input/data",
    s3_data_type = "S3Prefix",
    s3_input_mode = "File",
    s3_data_distribution_type = data_config.s3_data_distribution_type,
    s3_compression_type = data_config.s3_compression_type,
)

result_output = ProcessingOutput( 
    source = "/opt/ml/processing/output",
    destination = data_config.s3_output_path,
    output_name = "analysis_result",
    s3_upload_mode = "EndOfJob",
)

step_clarify = ProcessingStep(
    name = "ClarifyProcessingStep",
    processor = clarify_processor,
    inputs = [data_input, config_input],
    outputs = [result_output],
    depends_on = [step_config_file.name]
)

**RegisterChurnModel** 步骤是在名为 **step_register** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Register Model（注册模型）– 注册任务使用类 RegisterMode() 定义。
- **Estimator**（估算程序）：估算程序是在之前单元格中的 **xgbtrain** 变量中定义的。
- **Model Data**（模型数据）：这是 **ChurnHyperParameterTuning** 返回的模型 URI。
- **Content Types**（内容类型）：text/csv
- **Response Types**（响应类型）：text/csv
- **Inference Instance**（推理实例）：这是将用于推理处理的实例类型。
- **Transform Instance**（转换实例）：这是将用于处理转换的实例类型。
- **Model Package Group Name**（模型包组名称）：这是将存储模型版本组的组的名称。
- **Model Metrics**（模型指标）：这定义了模型指标的位置。其中包含的文件有 SageMaker Clarify 偏差报告、SageMaker Clarify 可解释性报告以及模型评估。

In [ ]:
#configure-model-registry
model_statistics = MetricsSource(
    s3_uri = "s3://{}/output/evaluation/evaluation.json".format(default_bucket),
    content_type = "application/json"
    )
explainability = MetricsSource(
    s3_uri = "s3://{}/clarify-output/bias/analysis.json".format(default_bucket),
    content_type = "application/json"
    )

bias = MetricsSource(
    s3_uri = "s3://{}/clarify-output/bias/analysis.json".format(default_bucket),
    content_type = "application/json"
    ) 

model_metrics = ModelMetrics(
    model_statistics = model_statistics,
    explainability = explainability,
    bias = bias
)

step_register = RegisterModel(
    name = "RegisterChurnModel",
    estimator = xgb_train,
    model_data = step_tuning.get_top_model_s3_uri(top_k = 0, s3_bucket = default_bucket, prefix = "output"),
    content_types = ["text/csv"],
    response_types = ["text/csv"],
    inference_instances = ["ml.t2.medium", "ml.m5.large"],
    transform_instances = ["ml.m5.large"],
    model_package_group_name = model_package_group_name,
    model_metrics = model_metrics,
)

**CheckAUCScoreChurnEvaluation** 步骤是在名为 **step_cond** 的变量中定义的。

步骤配置包括以下内容：
- **Type**（类型）：Condition（条件）– 条件任务使用类 ConditionStep() 进行定义。
- **Conditions**（条件）：如果 **ChurnEvalBestModel** 的输出大于 0.75，此条件将评估为 True（真）。
- **If Steps**（如果步骤）：这是当条件评估为 True（真）时将运行的步骤列表。
- **Else Steps**（否则步骤）：这是当条件评估为 False（假）时将运行的步骤列表。注意，此列表是空的，这意味着如果条件不满足，管道将停止处理。

In [ ]:
%%capture
cond_lte = ConditionGreaterThan(
    left = JsonGet(
        step = step_eval,
        property_file = evaluation_report,
        json_path = "binary_classification_metrics.auc.value"
    ),
    right = 0.75,
)

step_cond = ConditionStep(
    name = "CheckAUCScoreChurnEvaluation",
    conditions = [cond_lte],
    if_steps = [step_create_model, step_config_file, step_transform, step_clarify, step_register],
    else_steps = [],
)

### 任务 2.2.3 定义管道

定义完步骤后，在名为 **pipeline** 的变量中配置管道。注意之前定义的步骤是如何传递给管道定义的。

In [ ]:
 #define pipeline function
def get_pipeline(
    region,
    role = None,
    default_bucket = None,
    model_package_group_name = "ChurnModelPackageGroup",
    pipeline_name = "ChurnModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version = None
    ):

    #configure pipeline instance
    pipeline = Pipeline(
        name = pipeline_name,
        parameters = [
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps = [step_process, step_tuning, step_eval, step_cond],
        sagemaker_session = sagemaker_session
    )
    return pipeline


### 任务 2.2.4 创建管道

In [ ]:
 #create pipeline using function
pipeline = get_pipeline(
  region = region,
    role = role,
    default_bucket = default_bucket,
    model_package_group_name = model_package_group_name,
    pipeline_name = pipeline_name,
    custom_image_uri = clarify_image,
    sklearn_processor_version = sklearn_processor_version
)

### 任务 2.2.5 更新管道以使用正确的 IAM 角色

In [ ]:
#set-iam-role
pipeline.upsert(role_arn = role)

**注意**：如果在运行单元格后看到以下警告，可放心忽略。

<b>No finished training job found associated with this estimator.</b><b>Please make sure this estimator is only used for building workflow config.</b>（未找到与此估算器相关的已完成训练任务。请确保此估算器仅用于构建工作流配置。）

### 任务 2.2.6 开启管道

In [ ]:
#start-pipeline
RunPipeline = pipeline.start()

### 任务 2.2.7 描述管道

In [ ]:
#describe-pipeline
RunPipeline.describe()

运行此管道大约需要 35 分钟。

在管道运行期间，继续执行下一个任务以浏览 Amazon SageMaker Studio 控制台中的管道。

## 任务 2.3 监控并批准管道

在此任务中，您将使用 Amazon SageMaker Studio 控制台浏览管道。

### 任务 2.3.1 监控 SageMaker Studio 中的管道

下一个任务将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **lab_10.ipynb** 选项卡拖到一边，或者选择（右键单击）**lab_10.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览管道步骤时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完管道步骤后，通过选择 **lab_10.ipynb** 选项卡返回至笔记本。

1.在 SageMaker Studio 中，选择 **SageMaker 主页**图标。
1.选择 **Pipelines**（管道）。

此时 SageMaker Studio 将打开 **Pipelines**（管道）选项卡。

1.选择名为 **ChurnModelSMPipeline** 的管道。

此时 SageMaker Studio 将打开 **ChurnModelSMPipeline** 选项卡。

1.在 **ChurnModelSMPipeline** 选项卡的 **Executions**（执行）下，打开（右键单击）管道状态，然后选择 **Open execution details**（打开执行详细信息）。

此时 SageMaker Studio 将打开 **Directed Acyclic Graph** (DAG)（有向无环图）页面。

有向无环图 (DAG) 将显示管道的工作流和进度。颜色用于指示步骤状态。步骤颜色标志如下：
- **灰色**：等待运行。
- **蓝色**：正在运行。
- **绿色**：成功完成。
- **红色**：错误。

浏览的每个步骤都将有相同的四个选项卡显示在步骤的详情面板中。尽管选项卡标题相同，但每个选项卡的内容会因任务类型和任务配置而异：

- **Input**（输入）：此选项卡包含已传递给任务的输入。输入示例包括指定实例类型的参数、AWS Identity and Access Management (IAM) 角色或运行任务所需的参数。其他输入示例包括代码、数据集和 Docker 映像等文件。
- **Output**（输出）：此选项卡显示了任务创建的输出。输出的一些示例包括指标、图表、文件和评估结果。
- **Logs**（日志）：此选项卡提供了与任务相关的日志列表。如果用户具有访问 Amazon CloudWatch 日志的足够权限，可以在 CloudWatch 中选择日志链接并查看详细日志信息。某些任务类型不生成日志。
- **Information**（信息）：此选项卡提供了有关任务的基本信息，例如任务类型、任务名称和任务开始运行的时间。

1.选择名为 **ChurnModelProcess** 的步骤。此时将显示名为 **ChurnModelProcess** 的新窗格。
1.在 **ChurnModelProcess** 窗格中，查看与此管道步骤相关的选项卡： 
    - 选择 **Input**（输入）选项卡。此选项卡包含有关处理步骤使用的参数和文件的有用信息。参数列表中显示了详细信息，包括任务使用的实例类型和映像、数据集位置、代码位置以及生成的不同输出的目的地。滚动到窗格底部，找到已传递给任务的文件输入。
    - 选择 **Output**（输出）选项卡。此选项卡显示了管道步骤生成的不同文件及其位置。此管道将所有输出都放在 SageMaker Studio 默认存储桶。
    - 选择 **Logs**（日志）选项卡。此选项卡显示了任务生成的日志。启用 SageMaker Studio 内提供的日志记录功能，可加快执行管道步骤运行失败时的调查和故障排除操作。
    - 选择 **Information**（信息）选项卡。此选项卡简要概述了管道步骤，其中包括步骤类型、步骤名称和任务日志链接等信息。它还提供了有关任务开始运行的时间和运行耗时的详细信息。
        - 注意，**Step Type**（步骤类型）是 **Processing**（处理）。

### 任务 2.3.2 发现管道步骤详细信息

在以下步骤中，从有向无环图 (DAG) 中选择合适的节点，找到有关指定管道步骤的信息。如果在查找答案时需要帮助，您可以在此 Python 笔记本结尾找到正确的答案或提示。

1.对于名为 **ChurnHyperParameterTuning** 的步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？
    - 此步骤生成的 **Overall Best Training Job**（整体最佳训练任务）是什么？
1.对于名为 **ChurnEvalBestModel** 的步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？
    - 用于评估在之前步骤中确认的顶级模型的 Python 脚本叫什么？
    - 此文件在哪里？
    - 此步骤的结果写在哪个位置？
1.对于名为 **CheckAUCScoreChurnEvaluation** 的步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？
    - **Evaluation outcome**（评估结果）是什么？
1.对于名为 **ChurnCreateModel** 的步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？
    - 此任务是否生成了任何日志？
1.对于名为 **RegisterChurnModel** 的步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？
    - 曲线 (AUC) 指标下的区域的值是多少？
1.对于名为 **ChurnTransform** 的步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？
    - 此任务是否生成了日志？
    - 此步骤的输入有哪些文件？
1.对于名为 **ChurnModelConfigFile** 的步骤，找到以下详细信息： 
    - 用于运行此任务的是哪个 ProcessingInstanceType？
    - 此步骤的 **Step Type**（步骤类型）是什么？
1.对于名为 **ClarifyProcessingStep** 的步骤，找到以下详细信息：
    - 此步骤的文件输出是什么？
    - 输出写在哪个位置？

### 2.3.3 批准管道中的模型

1.管道完成运行后，查看管道在 **Model registry**（模型注册表）中创建的模型：
    - 在 SageMaker Studio 中，选择 **SageMaker 主页**图标。
    - 展开 **Models**（模型）列表。
    - 选择 **Model registry**（模型注册表）。
    - 打开名为 **ChurnModelPackageGroup** 的模型组。
    - 在 **ChurnModelPackageGroup** 选项卡中，打开（右键单击）**Versions**（版本）表中的行，然后选择 **Open model version**（打开模型版本）。注意，模型状态为 **Pending**（待处理）。另请注意，**Execution**（执行）值是刚完成的管道运行的名称。

    有关管道的其他详细信息可在每个选项卡中找到：
    - **Activity**（活动）：此选项卡显示了模型的活动。它包含事件信息以及自模型修改以来所经历的时间。
    - **Model quality**（模型质量）：此选项卡显示了模型准确性指标。
    - **Explainability**（可解释性）：此选项卡显示了模型特征在 Shapley 值 (SHAP) 方面的重要性。
    - **Bias report**（偏差报告）：此选项卡显示了潜在模型偏差。
    - **Inference recommender**（推理建议）：此选项卡提供了提升模型性价比的建议。此选项卡不包含数据，因为此模型包不支持此功能。
    - **Load test**（负载测试）：在此选项卡中，您可以启动负载测试，尝试不同的实例类型并就生产部署要求的所需吞吐量和延迟指标对其进行评估。
    - **Settings**（设置）：此选项卡显示了模型创建时间、生成模型的管道、模型所在的位置以及与模型有关的试用组件等信息。

1.批准模型。此流程的目的是在批准模型前进行手动审查。但是，也可以在管道内自动批准模型：
    - 选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**Update status**</span>（更新状态）。
    - 打开下拉列表，然后选择 <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">**Approved**</span>（已批准）。
    - 选择 <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**Update status**</span>（更新状态）。
1.选择 **ChurnModelPackageGroup** 选项卡。

### 任务 2.3.4 使用 AWS SDK 查看管道步骤

除了使用 SageMaker Studio UI 查看管道详细信息，您还可以使用 AWS SDK 命令。例如，以下命令可返回管道步骤列表。

In [ ]:
#list-steps
RunPipeline.list_steps()

## 任务 2.4 查看构件

下一个任务将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **lab_10.ipynb** 选项卡拖到一边，或者选择（右键单击）**lab_10.ipynb** 选项卡并选择 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览构件时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完构件后，通过选择 **lab_10.ipynb** 选项卡返回至笔记本。

### 任务 2.4.1 在 SageMaker Studio 中查看构件

在管道运行期间，每个步骤都会生成文件、经过训练的参数和模型等构件。您可以确定 SageMaker Studio 中管道创建的构件。
1.返回至名为 **ChurnModelSMPipeline** 的选项卡。
1.选择 **Executions**（执行）选项卡。
1.打开（右键单击）列出的执行，然后选择 **View trial components generated by execution**（查看执行生成的试用组件）。

此时 SageMaker Studio 将打开一个名为 **Trial Component List**（试用组件列表）的新选项卡。

系统将显示一个包含管道运行的所有任务的列表。

注意，每个试用组件都有 **Trial Component Type**（试用组件类型）。在相关试用详细信息的各种选项卡下提供的信息都取决于试用组件类型。并非试用详细信息下的所有选项卡都能使用所有组件类型的数据填充。

1.打开（右键单击）任务列表的顶行，然后选择 **Open in trial details**（在试用详细信息中打开）。

此时 SageMaker Studio 将打开一个名为 **Describe Trial Component**（描述试用组件）的新选项卡。

**Trial Components**（试用组件）下有多个可用选项卡。根据管道步骤执行的操作，一些选项卡可能显示为空。

1.选择 **Artifacts**（构件）选项卡。此选项卡将显示步骤中使用的输入和输出的详细信息。
1.选择 **Explainability**（可解释性）选项卡。此选项卡将显示 SageMaker Clarify 生成的可解释性报告。
1.选择 **Bias Report**（偏差报告）选项卡。此选项卡将显示 SageMaker Clarify 生成的偏差报告。

### 任务 2.4.2 找到默认 S3 存储桶中的构件
**注意**：您将使用 AWS 管理控制台执行此任务。浏览完 S3 存储桶后，返回至打开了 SageMaker Studio 的浏览器标签页，然后选择 **lab_10.ipynb** 选项卡。

1.在打开了控制台的浏览器标签页中，导航到 Amazon S3。
1.选择名称以 **sagemaker-** 和 AWS 区域开头的存储桶，例如 **sagemaker-us-west-2-123456789**。
1.浏览此存储桶下的文件夹和文件。此存储桶包含数据集、处理输入和输出、SageMaker Clarify 结果以及有助于生成模型的其他文件。
1.返回至打开了 SageMaker Studio 的浏览器标签页，然后选择 **lab_10.ipynb** 选项卡。

## 任务 2.5（可选）构建并查看管道沿袭

您学习了如何借助 SageMaker Clarify 来解释模型如何进行预测以及了解模型的潜在偏差。您还可以使用 SageMaker Clarify 发现用于生成模型的步骤，这在模型审计中经常用到。在此任务中，您将利用 MLLineageHelper 模块构建当前管道运行的沿袭。有关 ML 沿袭帮助程序的更多信息，请参阅 [MLLineageHelper](https://github.com/aws-samples/ml-lineage-helper)。

Amazon SageMaker ML 沿袭跟踪功能可创建和存储有关 ML 工作流步骤（从数据准备到模型部署）的信息。通过跟踪信息，您可以重现工作流步骤、跟踪模型和数据集沿袭以及建立模型监管和审计标准。

### 任务 2.5.1 设置会话和变量

In [ ]:
#set-variables
fs_query = feature_group.athena_query()
fs_table = fs_query.table_name
query_string = 'SELECT * FROM "'+fs_table+'"'

### 任务 2.5.2 显示将用于构建模型沿袭的值

配置包括以下各项：
- **query_string**：这是将传递给 MLLineageHelper 模块的 SageMaker 特征存储查询。
- **model_ref**：这是要评估的模型的名称。
- **processing_job**：这是生成模型的处理任务的名称。

In [ ]:
#print-values
print ('query_string:',query_string)

model_ref = sagemaker_client.list_models(SortBy = 'CreationTime', SortOrder = 'Descending')['Models'][0]['ModelName']
print ('model_ref:',model_ref)

processing_job = sagemaker_client.list_processing_jobs(SortBy = 'CreationTime', SortOrder = 'Descending', NameContains = 'ChurnModelProcess')['ProcessingJobSummaries'][0]['ProcessingJobName']
print ('processing_job:',processing_job)

processing_job_description = sagemaker_client.describe_processing_job(
    ProcessingJobName = processing_job
    )

### 任务 2.5.3 描述处理任务

In [ ]:
#describe-processing-job
processing_job_description

### 任务 2.5.4 显示用于创建模型的训练任务的名称

In [ ]:
#print-training-job
training_job_name  =  sagemaker_client.list_training_jobs(SortBy = 'CreationTime', SortOrder = 'Descending')['TrainingJobSummaries'][0]['TrainingJobName']
print (training_job_name)

### 任务 2.5.5 构建模型沿袭

如果收到以下错误消息，请再次运行单元格。
- **ClientError: An error occurred (ThrottlingException) when calling the UpdateArtifact operation (reached max retries: 4): Rate exceeded**（ClientError: 调用 UpdateArtifact 操作时发生错误 (ThrottlingException) (达到最大重试次数: 4): 已超出速率）

In [ ]:
#build-lineage
ml_lineage = MLLineageHelper()
lineage = ml_lineage.create_ml_lineage(training_job_name, model_name = model_ref,
                                       query = query_string, sagemaker_processing_job_description = processing_job_description,
                                       feature_group_names = [feature_group_name])

### 任务 2.5.6 限制沿袭以仅包括当前试用和特征组

管道可运行多次。为确保检索的是有关最新的训练任务运行的详细信息，请使用当前试用及其使用的特征组的名称筛选沿袭调用。

运行完此单元格后，用于创建模型的步骤、步骤运行顺序以及对管道中其他任务有帮助的任务将以表形式显示。同样的信息也将写入名为 **lineage_FS.csv** 的文件。您可以下载此文件以保存输出，然后将其分享给其他团队成员，例如审计人员。

In [ ]:
#limit-lineage
trial_name = RunPipeline.describe()['PipelineExperimentConfig']['TrialName']
pat = str(trial_name)+'|'+'fg-FG'
df1 = lineage[lineage.apply(lambda x: any(x.str.contains(pat)),axis = 1)]
pd.set_option('display.max_colwidth', 120)
df1.to_csv('lineage_FS.csv') 
df1

### 任务 2.5.7 生成可视化的模型沿袭

In [ ]:
#visualize-lineage
plt.figure(3, figsize = (20, 14))
graph = nx.DiGraph()
graph.add_edges_from([(each[0], each[2]) for each in df1.values])
fig, ax = plt.subplots()
nx.draw_networkx(
    graph,
    node_size = 300,
    node_color = "orange",
    alpha = 0.65,
    font_size = 8,
    pos = nx.spring_layout(graph)
)
ax.set_facecolor('deepskyblue')
ax.axis('off')
fig.set_facecolor('deepskyblue')
plt.show()

## 任务 2.6 删除管道

要删除管道，请运行以下单元格：

In [ ]:
#delete-pipeline
response = sagemaker_client.delete_pipeline(PipelineName = 'ChurnModelSMPipeline')
print (response)

### 总结 

恭喜！ 您已经使用 SageMaker 管道自动创建和注册模型。您学习了如何深入了解每个管道步骤，确定相关参数、文件和日志。您知道了如何确认管道用于生成模型的资产、如何在模型注册表中找到模型，以及如何找到并查看管道生成的可解释性报告和偏差报告。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续**总结**部分。

### 任务 2.3.1.1 的提示和答案
一般提示：**Step type**（步骤类型）位于 **Information**（信息）选项卡。

1.对于名为 **ChurnHyperParameterTuning** 的步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？</br>
    **答案**：Tuning（优化）</br>
    - 此步骤生成的 **Overall Best Training Job**（整体最佳训练任务）是什么？ </br>
    **提示**：此信息位于 **Output**（输出）选项卡。</br>
    **答案**：模型名称针对每个学员生成，且因人而异。名称应类似于以下示例：056vhzs2vkxc-ChurnHy-TCAtUr16oV-001-17d5bd01
1.对于 **ChurnEvalBestModel** 步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？
    **答案**：Processing（处理）</br>
    - 用于评估在之前步骤中确认的顶级模型的 Python 脚本叫什么？</br>
    **提示**：此信息位于 **Intput**（输入）选项卡。</br>
    **答案**：evaluate.py</br>
    - 此文件在哪里？</br>
    **提示**：此信息位于 **Intput**（输入）选项卡。</br>
    **答案**：此文件位于 S3 存储桶。路径类似于以下示例：s3://sagemaker-us-west-2-1234567890/input/code/evaluate.py</br>
    - 此步骤的结果写在哪个位置？</br>
    **提示**：此信息位于 **Output**（输出）选项卡。</br>
    **答案**：评估结果将写入 S3 存储桶。文件路径类似于以下示例：s3://sagemaker-us-west-2-1234567890/output/evaluation</br>
1.对于 **CheckAUCScoreChurnEvaluation** 步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？</br>
    **答案**：Condition（条件）</br>
    - **Evaluation outcome**（评估结果）是什么？</br>
    **提示**：此信息位于 **Output**（输出）选项卡。</br>
    **答案**：True（真）
1.对于 **ChurnCreateModel** 步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？</br>
    **答案**：Model（模型）</br>
    - 此任务是否生成了任何日志？</br>
    **答案**：否
1.对于 **RegisterChurnModel** 步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？</br>
    **答案**：RegisterModel（注册模型）</br>
    - AUC 指标的值是多少？</br>
    **提示**：此信息位于 **Output**（输出）选项卡。</br>
    **答案**：该值会变，但应接近 0.98。
1.对于 **ChurnTransform** 步骤，找到以下详细信息：
    - 此步骤的 **Step Type**（步骤类型）是什么？</br>
    **答案**：Transform（转换）</br>
    - 此任务是否生成了日志？</br>
    **答案**：是</br>
    - 此步骤的输入有哪些文件？</br>
    **提示**：此信息位于 **Intput**（输入）选项卡。您可能需要滚动到窗格底部才能找到文件名称。</br>
    **答案**：model.tar.gz、sagemaker-xgboost:1.5-1-cpu-py3 和 batch.csv
1.对于 **ChurnModelConfigFile** 步骤，找到以下详细信息： 
    - 用于运行此任务的是哪个 ProcessingInstanceType？</br>
    **提示**：此信息位于 **Intput**（输入）选项卡。</br>
    **答案**：ml.m5.xlarge
    - 此步骤的 **Step Type**（步骤类型）是什么？</br>
    **答案**：Processing（处理）
1.对于 **ClarifyProcessingStep** 步骤，找到以下详细信息：
    - 此步骤的文件输出是什么？
    **提示**：此信息位于 **Output**（输出）选项卡。</br>
    **答案**：输出是偏差数据。
    - 输出写在哪个位置？
    **答案**：输出将写入 S3 存储桶。路径类似于以下示例：s3://sagemaker-us-west-2-1234567890/clarify-output/bias